# Deep Fruit Vision Evaluation

Our detection, ripeness, and defect models are all trained and tested on different datasets. This notebook evaluates the overall performance of Deep Fruit Vision on a unique, hand-labelled test dataset.

In [1]:
import os
from modules.datasets import TestDataset, save_dataset
import matplotlib.pyplot as plt

In [2]:
detection_weights = os.path.join('weights', 'detection', 'best.pt')
ripeness_weights = os.path.join('weights', 'ripeness', 'ripeness_model')
defect_weights = os.path.join('weights', 'disease', 'defect_model')

We can use the TestDataset class to easily load images and labels from the test dataset.

In [3]:
test_dataset = TestDataset('dataset')

## Detection mAP

The detection/Yolo-v5 model is responsible for detecting and classifying fruits, so we can just use Yolo-v5's built-in evaluation script.

In [4]:
test_dataset.for_yolov5_eval = True

detection_eval_img_dir = os.path.join('dataset', 'yolov5_test', 'images')
detection_eval_label_dir = os.path.join('dataset', 'yolov5_test', 'labels')

os.makedirs(detection_eval_img_dir, exist_ok=True)
os.makedirs(detection_eval_label_dir, exist_ok=True)

save_dataset(test_dataset, detection_eval_img_dir, detection_eval_label_dir)

Saving dataset to dataset\yolov5_test\images and dataset\yolov5_test\labels:  31%|███       | 9/29 [00:00<00:00, 80.23it/s]

   class         x         y         w        h
0      1  0.503868  0.407806  0.992263  0.30148
   class         x         y         w         h
0      2  0.442482  0.502019  0.735621  0.864065
1      2  0.747730  0.240915  0.278507  0.398385
   class         x         y         w         h
0      2  0.397590  0.437416  0.333333  0.312248
1      2  0.564257  0.587483  0.297189  0.262450
   class         x         y         w         h
0      1  0.491921  0.503365  0.725314  0.969044
1      1  0.206463  0.162180  0.341113  0.324361
2      1  0.885996  0.180350  0.202873  0.358008
   class         x         y         w         h
0      2  0.485533  0.539031  0.786618  0.827725
   class         x         y         w         h
0      1  0.528708  0.378869  0.794258  0.641992
1      1  0.718900  0.732167  0.490431  0.255720
2      1  0.228469  0.611036  0.456938  0.358008
3      1  0.166268  0.170929  0.318182  0.341857
   class         x         y         w         h
0      2  0.587971  0.

Saving dataset to dataset\yolov5_test\images and dataset\yolov5_test\labels: 100%|██████████| 29/29 [00:00<00:00, 80.76it/s]

   class        x         y         w         h
0      2  0.48833  0.515478  0.603232  0.732167
   class         x         y         w         h
0      2  0.513475  0.517497  0.944681  0.927322
   class         x        y         w         h
0      1  0.565817  0.48183  0.575372  0.810229
   class         x         y         w         h
0      1  0.708880  0.502019  0.572149  0.982503
1      1  0.310797  0.576043  0.518668  0.837147
2      1  0.337538  0.153432  0.356206  0.296097
   class         x         y         w         h
0      1  0.631846  0.781965  0.115619  0.185734
1      1  0.617647  0.496635  0.123732  0.207268
2      1  0.552738  0.372140  0.103448  0.138627
3      1  0.431034  0.395020  0.115619  0.157470
4      1  0.382353  0.514805  0.099391  0.154778
5      1  0.347870  0.376851  0.103448  0.134590
   class         x         y         w         h
0      2  0.346499  0.646030  0.377020  0.384926
1      2  0.582585  0.632571  0.174147  0.242261
   class         x      

In [5]:
!python yolov5/val.py --data apple_papaya_mango.yaml --weights {detection_weights} --img 416 --task test

val: data=apple_papaya_mango.yaml, weights=['weights\\detection\\best.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5\runs\val, name=exp, exist_ok=False, half=False, dnn=False
fatal: cannot change to 'C:\Users\jorda\Documents\School\CMPE': No such file or directory
YOLOv5  2022-10-19 Python-3.10.5 torch-1.12.0 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs

test: Scanning 'C:\Users\jorda\Documents\School\CMPE 295\fruit-detection\dataset\yolov5_test\labels.cache' images and labels... 29 found, 0 missing, 0 empty, 0 corrupt: 100%|##########| 29/29 [00:00<?, ?it/s]
test: Scanning 'C:\Users\jorda\Documents\School\CMPE 295\fruit-detection\dataset\yolov5_test\labels.cache' images and labels... 29 found, 0 missing, 0 e

In [ ]:
# set this to be false now that we are done evaluating the detection model
test_dataset.for_yolov5_eval = False

## Ensemble Classification Accuracy

Now we want to evaluate the accuracy of the ensemble classification model.